In [30]:
import pandas as pd 
import ast

df = pd.read_csv("../datas/movies_metadata.csv")
df.genres = df.genres.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df.genres = df.genres.apply(lambda x: [genre['name'] for genre in x])
df = df.astype({'title':'string'})
df = df.dropna(subset=['title','overview'])
df2 = df[["title", "overview", "genres",'id']]
df['id'] = df['id'].astype('int64')

C:\Users\feldm\AppData\Local\Temp\ipykernel_5696\3339230785.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datas/movies_metadata.csv")


In [31]:

merged = pd.read_csv('../datas/merged_links.csv')
merged.head()

,movieId,imdbId,tmdbId,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,imdb_id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,114709,862.0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,tt0114709,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,2,113497,8844.0,1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,tt0113497,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,3,113228,15602.0,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,tt0113228,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,4,114885,31357.0,3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,tt0114885,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,5,113041,11862.0,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,tt0113041,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vecto():
    vectorize = TfidfVectorizer(ngram_range=(1,2))
    tfid = vectorize.fit_transform(merged['title'])
    return tfid

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

def search(title):
    query_vec = vectorize.transform([title])
    similarity = cosine_similarity(query_vec, tfid).flatten()
    indices = np.argpartition(similarity,-5)[-5:]
    results = merged.iloc[indices][::-1]
    display(results[['title', 'overview', 'release_date','genres']])
    return results['movieId'].iloc[0]

In [34]:
ratings = pd.read_csv('../datas/ratings.csv')


In [35]:
def find_similar_movie(movie_id, num_of_movie):
    # find users similar to us 
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings['rating'] > 4 )]['userId'].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings['rating'] > 4)]["movieId"]
    # adjusting recommendations ( more than 10% recommended the movie)
    similar_user_recs = similar_user_recs.value_counts() /len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .1]
    # find how common the recommendation where over all users 
    all_user = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_user["movieId"].value_counts() / len(all_user["userId"].unique())
    # creation of a score for recommendation
    rec_percentages = pd.concat([similar_user_recs,all_users_recs], axis=1)
    rec_percentages.columns = ['similar', 'all']
    rec_percentages["score"] = rec_percentages['similar'] / rec_percentages["all"]
    # sorting the score to have the more logical on top 
    rec_percentages = rec_percentages.sort_values("score", ascending= False)
    return rec_percentages.head(num_of_movie).merge(merged, left_index=True, right_on='movieId')[['score','title','genres','release_date']]



In [36]:
id = search('Harry Potter')
find_similar_movie(id, 10)

,title,overview,release_date,genres
4763,Harry Potter and the Philosopher's Stone,Harry Potter has lived under the stairs at his...,2001-11-16,"['Adventure', 'Fantasy', 'Family']"
10563,Harry Potter and the Goblet of Fire,"Harry starts his fourth year at Hogwarts, comp...",2005-11-05,"['Adventure', 'Fantasy', 'Family']"
5680,Harry Potter and the Chamber of Secrets,"Ignoring threats to his life, Harry returns to...",2002-11-13,"['Adventure', 'Fantasy', 'Family']"
11939,Harry Potter and the Order of the Phoenix,Returning for his fifth year of study at Hogwa...,2007-06-28,"['Adventure', 'Fantasy', 'Family', 'Mystery']"
7733,Harry Potter and the Prisoner of Azkaban,"Harry, Ron and Hermione return to Hogwarts for...",2004-05-31,"['Adventure', 'Fantasy', 'Family']"


,score,title,genres,release_date
4763,31.436796,Harry Potter and the Philosopher's Stone,"['Adventure', 'Fantasy', 'Family']",2001-11-16
5680,23.011760,Harry Potter and the Chamber of Secrets,"['Adventure', 'Fantasy', 'Family']",2002-11-13
11939,19.367161,Harry Potter and the Order of the Phoenix,"['Adventure', 'Fantasy', 'Family', 'Mystery']",2007-06-28
10563,19.085719,Harry Potter and the Goblet of Fire,"['Adventure', 'Fantasy', 'Family']",2005-11-05
13904,17.688142,Harry Potter and the Half-Blood Prince,"['Adventure', 'Fantasy', 'Family']",2009-07-07
7733,17.303855,Harry Potter and the Prisoner of Azkaban,"['Adventure', 'Fantasy', 'Family']",2004-05-31
16122,15.733192,Harry Potter and the Deathly Hallows: Part 1,"['Adventure', 'Fantasy', 'Family']",2010-10-17
17413,14.748663,Harry Potter and the Deathly Hallows: Part 2,"['Family', 'Fantasy', 'Adventure']",2011-07-07
10606,12.650106,"The Chronicles of Narnia: The Lion, the Witch ...","['Adventure', 'Family', 'Fantasy']",2005-12-07
11839,11.042479,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']",2007-05-19


In [9]:
import pandas as pd
df = pd.read_csv('../cleaned_data/cleaned_movies.csv')
df = df.set_index('movieId')
df.loc[1]

imdbId                                                             114709
tmdbId                                                              862.0
adult                                                               False
budget                                                           30000000
genres                                  ['Animation', 'Comedy', 'Family']
imdb_id                                                         tt0114709
original_language                                                      en
overview                Led by Woody, Andy's toys live happily in his ...
popularity                                                      21.946943
poster_path                              /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies       [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries    [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                   1995-10-30
revenue                               